# StressGait Analysis - Saliva & Self-Reports

## Setup and Helper Functions

In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import pingouin as pg
import seaborn as sns
from biopsykit.stats import StatsPipeline
from fau_colors import cmaps, register_fausans_font

from stressgait_analysis.dataset import StressGaitDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties_light)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

In [ ]:
deploy_type = "local"

config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
base_path

In [ ]:
export_path = Path("../../exports")
plot_path = export_path.joinpath("plots")
bp.utils.file_handling.mkdirs([export_path, plot_path])

export_path

In [ ]:
dataset = StressGaitDataset(base_path, coarse_condition=True)

dataset

In [ ]:
order = ["control", "omc"]
hue_order = ["control", "omc"]

In [ ]:
dataset.cortisol.unstack().head()

In [ ]:
s0s1_inc = dataset.cortisol.reindex(["S0", "S1"], level="sample").unstack().diff(axis=1).dropna(axis=1, how="all")
s0s1_inc_percent = (s0s1_inc / dataset.cortisol.xs("S0", level="sample").values) * 100

s0s1_inc.columns = pd.MultiIndex.from_tuples([("cortisol", "inc_S0S1")], names=[None, "saliva_feature"])
s0s1_inc_percent.columns = pd.MultiIndex.from_tuples([("cortisol", "inc_S0S1_percent")], names=[None, "saliva_feature"])

s0s1_inc.head()

In [ ]:
cortisol_features = dataset.cortisol_features.unstack().join(s0s1_inc).join(s0s1_inc_percent).stack(future_stack=True)
cortisol_features.head()

In [ ]:
cortisol_features.groupby(["condition", "saliva_feature"]).mean().unstack("condition")

In [ ]:
cortisol_normalized = (dataset.cortisol.unstack() - dataset.cortisol.unstack()[[("cortisol", "S0")]].values).stack(
    future_stack=True
)
cortisol_normalized.head()

In [ ]:
fig, ax = plt.subplots()

bp.protocols.plotting.saliva_plot(
    data=dataset.cortisol,
    saliva_type="cortisol",
    sample_times=dataset.sample_times,
    test_times=[30, 38],
    test_title="GAIT",
    sample_times_absolute=True,
    ax=ax,
)

fig.tight_layout()
fig.savefig(plot_path.joinpath("img_cortisol_response.pdf"), transparent=True)

In [ ]:
fig, ax = plt.subplots()

bp.protocols.plotting.saliva_plot(
    data=cortisol_normalized,
    saliva_type="cortisol",
    sample_times=dataset.sample_times,
    test_times=[30, 38],
    test_title="GAIT",
    sample_times_absolute=True,
    ax=ax,
)

ax.set_ylabel("Cortisol Increase [nmol/l]")

fig.tight_layout()
fig.savefig(plot_path.joinpath("img_cortisol_response_normalized.pdf"), transparent=True)

In [ ]:
fig, ax = plt.subplots()

sns.boxplot(data=dataset.cortisol.reset_index(), x="sample", y="cortisol", hue="condition", hue_order=hue_order, ax=ax)
sns.swarmplot(
    data=dataset.cortisol.reset_index(),
    x="sample",
    y="cortisol",
    hue="condition",
    hue_order=hue_order,
    ax=ax,
    palette=cmaps.faculties[:2],
    dodge=True,
    legend=False,
)

fig.tight_layout()
fig.savefig(plot_path.joinpath("img_boxplot_cortisol_samples.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(figsize=(14, 5), ncols=2, sharey=True)

dataset.cortisol.unstack()["cortisol"].xs("control", level="condition").T.plot(ax=axs[0], legend=False, title="Control")
dataset.cortisol.unstack()["cortisol"].xs("omc", level="condition").T.plot(ax=axs[1], legend=False, title="OMC")

# sns.lineplot(data=dataset.cortisol.reset_index(), x="sample", y="cortisol", ax=ax)

fig.tight_layout()

In [ ]:
pg.pairwise_tests(
    data=cortisol_features.xs("inc_S0S1", level="saliva_feature").reset_index(), dv="cortisol", between="condition"
)

In [ ]:
fig, axs = plt.subplots(ncols=4)

for key, ax in zip(["inc_S0S1", "inc_S0S1_percent", "max_inc_percent", "auc_i"], axs, strict=False):
    data_slice = cortisol_features.xs(key, level="saliva_feature")
    sns.boxplot(
        data=data_slice.reset_index(), x="condition", y="cortisol", hue="condition", order=order, ax=ax, showmeans=True
    )

    ax.set_title(key)

    test_results = pg.pairwise_tests(data=data_slice.reset_index(), dv="cortisol", between="condition")
    display(key)
    display(test_results)

fig.tight_layout()
# fig.savefig(plot_path.joinpath("img_boxplot_cortisol_samples.pdf"), transparent=True)

In [ ]:
pipeline = StatsPipeline(
    steps=[("test", "mixed_anova"), ("posthoc", "pairwise_tests")],
    params={
        "dv": "cortisol",
        "subject": "subject",
        "within": "sample",
        "between": "condition",
        "multicomp": {"method": "bonf", "levels": True},
    },
)

pipeline.apply(dataset.cortisol)

# box_pairs, pvalues = pipeline.sig_brackets("posthoc", plot_type="single", x="subscale", subplots=True, stats_effect_type="within")

pipeline.display_results(sig_only=False)

## Responder

In [ ]:
responder_list = cortisol_features.xs("inc_S0S1", level="saliva_feature") > 1.5

# display(responder_list)

pd.DataFrame(responder_list["cortisol"].groupby("condition").value_counts())

In [ ]:
responder_list = cortisol_features.xs("max_inc_percent", level="saliva_feature") > 1.5

# display(responder_list)

pd.DataFrame(responder_list["cortisol"].groupby("condition").value_counts())